**Zadanie 3.** 7 Wygeneruj dla słów formy superbazowe (patrz lista ćwiczeniowa 1). Zdefiniujemy
formę superbazową dla frazy jako połączenie form superbazowych dla składników tej frazy. Czyli
dla frazy:

> piłem wino na barce

forma superbazowa to na przykład

> pić|picie|piła wino|wina na barka|bark|barek

Napisz program, który przegląda 2 i 3-gramy z Narodowego Korpusu Języka Polskiego (linki
znajdziesz na SKOS) i znajduje sytuacje, w których tę samą formę superbazową mają frazy, w których na pewnej pozycji są wyrazy nie posiadające wspólnego lematu (jeżeli chcesz, możesz ograniczyć
się do N-gramów częstszych niż K). Przykładowo

> **piłą** win na **bark**
> 
> **piła** wino na **barce**

In [1]:
import csv
import logging
from collections import defaultdict

In [2]:
logging.basicConfig(level=logging.INFO)

In [3]:
BASE_FORMS_FILE_PATH = 'data/polimorfologik-2.1.txt'
BIGRAMS_FILE_PATH = 'data/poleval_2grams.txt'
TRIGRAMS_FILE_PATH = 'data/poleval_3grams.txt'

In [4]:
BASE_FORMS = defaultdict(set)

with open(BASE_FORMS_FILE_PATH) as f:
    for base_form, word, *_ in csv.reader(f, delimiter=';'):
        BASE_FORMS[word].add(base_form)

# Superbase forms

In [5]:
class SuperbaseForms:
    """A domain-specific disjoint set implementation"""

    class Element:
        
        def __init__(self, base_form):
            self.base_form = base_form
            self.base_forms = {base_form}
            self.parent = self
            self.rank = 0
        
        def __str__(self):
            return '|'.join(self.base_forms)
        
        def __repr__(self):
            return f'<SuperbaseForm {str(self)}>'
        
        def __eq__(self, other):
            return self.base_forms == other.base_forms
        
        def __len__(self):
            return len(self.base_forms)
        
        def __hash__(self):
            return hash(str(self))
        
        def hook_up_to(self, new_parent):
            self.parent = new_parent
            new_parent.base_forms.update(self.base_forms)
            if self.rank == self.parent.rank:
                self.parent.rank += 1
    
    def __init__(self):
        self._elements = {}
        
    def make_set(self, base_form):
        self._elements[base_form] = self.Element(base_form)
    
    def find(self, base_form):
        el = self._elements.get(base_form)
        if el is None:
            return None
        if el.parent is el:
            return el
        el.parent = self.find(el.parent.base_form)
        return el.parent
    
    def union(self, base_form_1, base_form_2):
        el1 = self.find(base_form_1)
        el2 = self.find(base_form_2)
        if el1.rank < el2.rank:
            el1.hook_up_to(el2)
        else:
            el2.hook_up_to(el1)

In [6]:
_SUPERBASE_FORMS = SuperbaseForms()

for word, [base_form, *more_base_forms] in BASE_FORMS.items():
    if _SUPERBASE_FORMS.find(base_form) is None:
        _SUPERBASE_FORMS.make_set(base_form)
    for another in more_base_forms:
        if _SUPERBASE_FORMS.find(another) is None:
            _SUPERBASE_FORMS.make_set(another)
        _SUPERBASE_FORMS.union(base_form, another)

def superbase_form(word):
    try:
        return _SUPERBASE_FORMS.find(next(iter(BASE_FORMS[word])))
    except StopIteration:
        raise ValueError(f'No superbase form for "{word}"')

# N-grams

In [7]:
def bold(text):
    return f'\033[1m{text}\033[m'

In [8]:
def display(superbase_form, ngrams, positions):
    ngram_rows = [
        ' '.join((bold(word) if i in positions else word) for i, word in enumerate(ng))
        for ng in ngrams
    ]
    print(' '.join(str(sf) for sf in superbase_form), *ngram_rows, sep='\n', end='\n\n')

In [9]:
def all_disjoint(sets):
    return all(
        s1.isdisjoint(s2)
        for i, s1 in enumerate(sets)
        for s2 in sets[i + 1 :]
    )

In [10]:
NGRAMS_BY_SUPERBASE_FORM = defaultdict(list)

def load_ngrams_from_file(file_path, max_num_ngrams=None):
    with open(file_path) as f:
        for i, line in enumerate(f):
            _freq, *ngram = line.split()
            try:
                sf = tuple(superbase_form(word) for word in ngram)
                NGRAMS_BY_SUPERBASE_FORM[sf].append(ngram)
            except ValueError:
                pass
            if i > 0 and i % 1_000_000 == 0:
                logging.info(f'Loaded {i} n-grams')
            if max_num_ngrams and i > max_num_ngrams:
                break

In [11]:
def find_ngrams_with_disjoint_base_forms(k=3):
    for superbase_form, ngrams in NGRAMS_BY_SUPERBASE_FORM.items():
        if len(ngrams) < k:
            continue
        positions = []
        for position in range(len(ngrams[0])):
            if all_disjoint([BASE_FORMS[ng[position]] for ng in ngrams]):
                positions.append(position)
        if positions:
            display(superbase_form, ngrams, positions)

In [13]:
load_ngrams_from_file(BIGRAMS_FILE_PATH)

INFO:root:Loaded 1000000 n-grams
INFO:root:Loaded 2000000 n-grams
INFO:root:Loaded 3000000 n-grams
INFO:root:Loaded 4000000 n-grams
INFO:root:Loaded 5000000 n-grams
INFO:root:Loaded 6000000 n-grams
INFO:root:Loaded 7000000 n-grams
INFO:root:Loaded 8000000 n-grams
INFO:root:Loaded 9000000 n-grams
INFO:root:Loaded 10000000 n-grams
INFO:root:Loaded 11000000 n-grams
INFO:root:Loaded 12000000 n-grams
INFO:root:Loaded 13000000 n-grams
INFO:root:Loaded 14000000 n-grams
INFO:root:Loaded 15000000 n-grams
INFO:root:Loaded 16000000 n-grams
INFO:root:Loaded 17000000 n-grams
INFO:root:Loaded 18000000 n-grams
INFO:root:Loaded 19000000 n-grams
INFO:root:Loaded 20000000 n-grams
INFO:root:Loaded 21000000 n-grams
INFO:root:Loaded 22000000 n-grams
INFO:root:Loaded 23000000 n-grams
INFO:root:Loaded 24000000 n-grams
INFO:root:Loaded 25000000 n-grams
INFO:root:Loaded 26000000 n-grams
INFO:root:Loaded 27000000 n-grams
INFO:root:Loaded 28000000 n-grams
INFO:root:Loaded 29000000 n-grams
INFO:root:Loaded 300000

In [14]:
find_ngrams_with_disjoint_base_forms()

jadalny|jadalnia dział|dzieje|dzielenie|dzielić|dziany|niedziany|dziać|działo|działać|działowy|działanie
jadalne dzielą
jadalnia działa
jadalnych działaniom

zapas|zapasy|zapasowy|zapadni|zapadnięcie|zapadły|zapaść|zapadnia wzgląd|względy|względem
zapadły względem
zapasowej względem
zapasów względem

choć rysi|rysunki|rysunkowy|rysunkowo|ryś|rysiowy|rys|rysa|rysy|rysunek|rysią
choć rysy
choć ryś
choć rysunki

umleć|umieć|umieść|umowy|umieścić|um|umowa|umówić|umielić wojo|województwo|wój|woj
umieszczenie województw
umowy województwo
um województwa

białogłowa|białogłowy stojący|stanowić|stać|stanowo|stojąco|stan|stały|niestojący|stal|stale|stała|stojąca|stoa|stanowy|stanąć|niestanie
białogłowy stanowiły
białogłowę stać
białogłowa stale

start|startowy|startowe|niestarty|zetrzeć|starcie operatorowy|operator
startowy operatora
starć operatorów
startem operator

dusząco|dusz|więcej|duszący|duży|duś|dusić|dusza|duha|nieduszący|dużo|najwięcej|wielki chrzest
więcej chrztów
największy chrzest


matowy|mamcia|mielić|mailowo|niemieć|matać|niemieniony|mail|memowy|mieść|miotła|me|maj|moje|mat|miotło|moi|majowe|miałowy|niemielony|maić|mailowy|maisz|mielone|miano|mama|mieścić|mem|mój|majowy|majowo|mata|mielony|mienie|mleć|maja|moja|mamić|miecić|miał|matema|mienić|mieniony|mać|miana|matowo|mieć chromosom|chromosomowy
mianem chromosomu
matach chromosomów
ma chromosomy

się gruchanie|gruch|grusza|gruchać|grucha|gruchania
siebie gruchają
się gruchą
się gruszę

zmiana minimum|minia|mina|minąć|minima|minuta|minięcie
zmianą minimum
zmianie minuty
zmian minął

staja|staj|staw|stawowa|stawowy|stajanie|stajowy|stawać|stawić|stawa|stawowo|stajać|staje|stajały oddziałowy|oddziałowa|oddział
stawu oddziały
stawiły oddziały
stają oddziały

namysł stojący|stanowić|stać|stanowo|stojąco|stan|stały|niestojący|stal|stale|stała|stojąca|stoa|stanowy|stanąć|niestanie
namyśle stał
namysłów stanowi
namyśle stanął

bieżący|barowy|bieżeć|bardzo|być|były|byt|barowo|niebytowy|bar|była|bytowy|niebieżący|niebyt|

przyglądać stojący|stanowić|stać|stanowo|stojąco|stan|stały|niestojący|stal|stale|stała|stojąca|stoa|stanowy|stanąć|niestanie
przyglądali stale
przyglądać stanowi
przygląda stojąca

skała|skala|skalić|nieskalany|skaleń|skalać|nieskalanie|skaleniowy imienie|imię
skalę imiona
nieskalane imię
skały imieniem

start|startowy|startowe|niestarty|zetrzeć|starcie niedokonany|dokonanie|dokonany|dokonać
startu dokonano
starć dokonali
startowych dokonano

tok|tokowy|toki starający|staro|starszy|starająca|stare|stara|stary|staranie|star|starać|starsza|starzeć|niestarający|starzy|starzyć|starzenie|najstarszy
toki starzenia
toku starano
toku starań

skurcz|skurczowy|skurczyć narządowy|narząd
skurczenie narządów
skurcze narządów
skurczowych narządów

seria|serio centr|centra|centralne|centrowy|center|centralny|centrowo|centrum
serii centralnych
serię centro
serii centrum

cios|cieszyć|ciesać|cieszenie|ciosa|ciosowy|ciosanie|ciosać góra|góry
cieszyć górą
ciosowej górze
ciosach górą

topić|topowy|top ko

cotygodniowo|cotygodniowy gromowy|gram|grana|godzina|gramatyk|gramatyka|grań|granowy|gra|gromowo|gromy|grom|gromić|grat|grać|gran|gramowy|granie
cotygodniowej godziny
cotygodniowego grania
cotygodniową grę

lekowy|lekarz|lek osiedlać|osiąść|osi|osa|osiać|oś|osiadły|osiedlić|osiedle|osieść
lekarza osiadłego
lekarz osiedlił
leku osiądzie

więzić|więzienie|więź|więźniowy|więznąć|więzień|wiąznąć zeznanie|zeznać|zeznawać
więźniu zeznań
więzień zeznaje
więźniów zeznał

wiedzieć|wieścić|wiedźmi|wieszczy|wieść|wieszczyć|wiedźma|wiedza|wieszczo|wieszcz|wieszczenie niezwykle
wiodą niezwykle
wiedza niezwykle
wiemy niezwykle

temat|tematowy|tematowo minimum|minia|mina|minąć|minima|minuta|minięcie
temat miny
temacie minęło
temat minimum
tematowej minuty

masło stojący|stanowić|stać|stanowo|stojąco|stan|stały|niestojący|stal|stale|stała|stojąca|stoa|stanowy|stanąć|niestanie
masłem stanowiła
masło stale
masła staliśmy

odnośnie nierzeczony|rzeknąć|rzec|rzeka|rzeczony|rzecz
odnośnie rzek
odnośnie rzec

on|jadło|im|jet|go|nie|nim|mu|imiesłów|imienia|jeść|jej|jedla|imiesłowowy|jedzenie endoskopowo|endoskop|endoskopowy|endoskopia
on endoskop
jej endoskopowego
mu endoskopię

toteż gromowy|gram|grana|godzina|gramatyk|gramatyka|grań|granowy|gra|gromowo|gromy|grom|gromić|grat|grać|gran|gramowy|granie
toteż grę
toteż godziny
toteż grań

pomnik|pomnikowo|pomnikowy dział|dzieje|dzielenie|dzielić|dziany|niedziany|dziać|działo|działać|działowy|działanie
pomnika działa
pomniki dzielą
pomnikiem dziejów

punktowy|pan|punkt|punktowo|pani|panie|piętro|patrz niedaleko
piętrze niedaleko
pani niedaleko
pan niedaleko

obliczyć|obliczać|obliczenie|oblicze|nieobliczony mnożenie|mnożyć
obliczamy mnożąc
obliczono mnożąc
obliczu mnożących

obliczyć|obliczać|obliczenie|oblicze|nieobliczony przyrość|przyrostowy|przyróść|przyrosły|przyrosnąć|przyrost
obliczeń przyrostu
obliczania przyrostu
obliczyć przyrost

umleć|umieć|umieść|umowy|umieścić|um|umowa|umówić|umielić efektowy|efekt|efekta
umieścimy efekty
umówione

...

punktowy|pan|punkt|punktowo|pani|panie|piętro|patrz korygować
punktów korygował
pani koryguje
pan korygować

porać|poraić|porównaj|por|pora|porowy niepalący|palenie|pałający|niepałający|palto|niepalenie|palący|pali|palowe|pałasz|niepaląca|pała|palma|paląca|pal|palić|pałać|palmowy|paląco|palowy|palm|pałająco|pala|pał
pory palmę
pory pal
pory pała

zarazem testowo|teścia|teściowie|test|teściowy|testowy|teściowa|teść
zarazem teściową
zarazem teściem
zarazem testem

kolęda stojący|stanowić|stać|stanowo|stojąco|stan|stały|niestojący|stal|stale|stała|stojąca|stoa|stanowy|stanąć|niestanie
kolęda stała
kolędy stanu
kolęda stanowiła

punktowy|pan|punkt|punktowo|pani|panie|piętro|patrz podłączyć|podłączać|podłącze
punkty podłączenia
panu podłączyć
pani podłączyła

woli|niewolący|niewoli|niewolić|woleć|wołać|wołający|wołająca|wole|wolej|wola|wołowy|wół|wołowe|niewola|niewolenie|wołanie|niewołający|niewoląca spory|sporzyć|spora|sporo|spór
wołał sporo
woli sporą
niewoli sporo

typ|typowo|typowy sło

trącać|trzy|tarło|trója|troić|trzecia|trzeć|tarta|tarcie|trzeci|trzem|trącić zarodek|zarodkowy
trzech zarodków
tartej zarodków
trzeci zarodek

osiedlać|osiąść|osi|osa|osiać|oś|osiadły|osiedlić|osiedle|osieść aż
osiadł aż
osą aż
osi aż
osiedla aż

zapewniać węgieł|węglany|węglić|węglan|węgiel|węgleć|węglowy|węgłowy|węglanowy
zapewniała węglowa
zapewniający węgiel
zapewnia węglan

środowisko niewystarczająco|wystarczająco|wystarczać|niewystarczający|wystarczający
środowiska niewystarczająco
środowisko wystarczająco
środowisko wystarcza

matowy|mamcia|mielić|mailowo|niemieć|matać|niemieniony|mail|memowy|mieść|miotła|me|maj|moje|mat|miotło|moi|majowe|miałowy|niemielony|maić|mailowy|maisz|mielone|miano|mama|mieścić|mem|mój|majowy|majowo|mata|mielony|mienie|mleć|maja|moja|mamić|miecić|miał|matema|mienić|mieniony|mać|miana|matowo|mieć cebulka
mają cebulki
mianem cebulek
moim cebulkom

niebojący|balowo|bal|balista|bać|boja|bania|bojowo|balowy|ban|boj|bojowy|bój nazywać
bojowych nazywane
boimy 

kołowe|kołowy|kolacho|kolega|koło|kołowa|kole|koł|kola|kołowo|koła|kół zrezygnować|niezrezygnowany|niezrezygnowanie|zrezygnowany|zrezygnowanie
kół zrezygnował
kolegami zrezygnowała
koli zrezygnowano

tradycyjny galowy|galić|galon|gala|gal|galowo|gała|galena|galonowy
tradycyjne gale
tradycyjny galowy
tradycyjny galon

tłuścieć|tłuszczowy|tłuścić|tłuste|tłusto|tłuszcza|tłusty|tłuszcz masy|masa
tłuste masy
tłuszczu masa
tłuszczowej masy

dusząco|dusz|więcej|duszący|duży|duś|dusić|dusza|duha|nieduszący|dużo|najwięcej|wielki definiować
duszę definiował
więcej definiuje
wielka definiowana

bin dać|niedany|dań|dany|danie|dane|da|dana|dacia|dan|dal|dama|data
bin dan
bin dato
bin da

sklepowa|sklep|sklepać|sklepowy|sklepić|sklepienie oświetlenie|oświetlić
sklepowe oświetlone
sklepy oświetlenie
sklepienie oświetlone

przy objawienie|objawić|objaw|objawowy|objawowo
przy objawach
przy objawowym
przy objawieniom

stojący|stanowić|stać|stanowo|stojąco|stan|stały|niestojący|stal|stale|stała|stojąca|s

umleć|umieć|umieść|umowy|umieścić|um|umowa|umówić|umielić lekko|lekki|lekce
umieszczono lekką
umieli lekkim
umowy lekko

powiedzieć|powierzchni|powiat|powiatowy|powiedzenie|powierzchny|powierzchnia|powieść spod
powierzchnię spod
powieści spod
powiatu spod

centr|centra|centralne|centrowy|center|centralny|centrowo|centrum la
centralnego la
centrum la
center la

pusta|puścić|puszczać|puszcza|puste|pusto|pusty|puścieć|puszczanin spory|sporzyć|spora|sporo|spór
puste spory
puszcza spory
puścili sporo

legnąć|lego|ląc|lecieć|leca|lęgnąć|legato|legat|lec|legło|legły|lecenie|legać|lęknąć|lega|lęgnia|legalista poprzez
leci poprzez
legato poprzez
lego poprzez

punktowy|pan|punkt|punktowo|pani|panie|piętro|patrz wyszczególnić|wyszczególnienie
pan wyszczególnić
punkty wyszczególnione
pani wyszczególniła

towar|towarowo|towarowy pasza|padły|pasat|padnięcie|niepadnięty|pasy|paść|pasatowy|pasowo|pas|pasowy|pasta|pasmo|padnięty|pach|padnąć|pachowy|pacha|padło|pasać
towar padła
towar pach
towaru pas

k

zasada osiedlać|osiąść|osi|osa|osiać|oś|osiadły|osiedlić|osiedle|osieść
zasady osiedlania
zasadzie osią
zasadzie osiedli

stojący|stanowić|stać|stanowo|stojąco|stan|stały|niestojący|stal|stale|stała|stojąca|stoa|stanowy|stanąć|niestanie mundur|mundurowy
stan mundur
stanowią mundurowi
stoją mundurowi

udo|udać|udany|nieudanie|udowy|udanie|udały|nieudany|ud wynosić
nieudanych wynoszącym
udo wynosi
udowej wynosi

środowiskowy|środowiskowo punktowy|pan|punkt|punktowo|pani|panie|piętro|patrz
środowiskowej pani
środowiskowego punktu
środowiskowych pan

dyskryminacja punktowy|pan|punkt|punktowo|pani|panie|piętro|patrz
dyskryminacji punktów
dyskryminacji pani
dyskryminacji pana

usytuowanie|usytuować stojący|stanowić|stać|stanowo|stojąco|stan|stały|niestojący|stal|stale|stała|stojąca|stoa|stanowy|stanąć|niestanie
usytuowanie stanowi
usytuowano stałą
usytuowania stoi

legnąć|lego|ląc|lecieć|leca|lęgnąć|legato|legat|lec|legło|legły|lecenie|legać|lęknąć|lega|lęgnia|legalista otrzymać
leca otrzyma

In [12]:
load_ngrams_from_file(TRIGRAMS_FILE_PATH, max_num_ngrams=40_000_000)

INFO:root:Loaded 1000000 n-grams
INFO:root:Loaded 2000000 n-grams
INFO:root:Loaded 3000000 n-grams
INFO:root:Loaded 4000000 n-grams
INFO:root:Loaded 5000000 n-grams
INFO:root:Loaded 6000000 n-grams
INFO:root:Loaded 7000000 n-grams
INFO:root:Loaded 8000000 n-grams
INFO:root:Loaded 9000000 n-grams
INFO:root:Loaded 10000000 n-grams
INFO:root:Loaded 11000000 n-grams
INFO:root:Loaded 12000000 n-grams
INFO:root:Loaded 13000000 n-grams
INFO:root:Loaded 14000000 n-grams
INFO:root:Loaded 15000000 n-grams
INFO:root:Loaded 16000000 n-grams
INFO:root:Loaded 17000000 n-grams
INFO:root:Loaded 18000000 n-grams
INFO:root:Loaded 19000000 n-grams
INFO:root:Loaded 20000000 n-grams
INFO:root:Loaded 21000000 n-grams
INFO:root:Loaded 22000000 n-grams
INFO:root:Loaded 23000000 n-grams
INFO:root:Loaded 24000000 n-grams
INFO:root:Loaded 25000000 n-grams
INFO:root:Loaded 26000000 n-grams
INFO:root:Loaded 27000000 n-grams
INFO:root:Loaded 28000000 n-grams
INFO:root:Loaded 29000000 n-grams
INFO:root:Loaded 300000

KeyboardInterrupt: 

In [15]:
find_ngrams_with_disjoint_base_forms(4)

stoa|stojąco|stanowo|stan|stojąca|stanowić|niestanie|stal|stale|stanąć|stała|stojący|stanowy|stać|niestojący|stały pochodzenie|pochodzić z
stałych pochodzących ze
stal pochodziła ze
stoi pochodzący z
stan pochodzi z

tylko jednia|jedno|jednane|jednać|niejednany|jeden|jednany punktowo|pan|patrz|punktowy|pani|panie|punkt|piętro
tylko jeden punkt
tylko jednego pan
tylko jedno piętro
tylko jedna pani

niebojący|balowo|balista|boj|bojowo|boja|bój|bal|bać|bania|balowy|ban|bojowy i wieko|wiekowo|w|wiek|wiekowy
bojowej i w
boju i w
boimy i w
balach i w

powiedzenie|powierzchni|powierzchnia|powieść|powierzchny|powiatowy|powiat|powiedzieć aż do
powieści aż do
powierzchni aż do
powiaty aż do
powiatowej aż do

i brakowy|brak|braki bar|bary|niebytowy|niebyt|być|barowo|bytowy|bieżąco|bytowo|bycie|była|bardzo|bieżący|barowy|bądź|były|niebieżący|byt|bieżeć
i brak bieżącej
i brak bardzo
i brak będzie
i brak baru

i bar|bary|niebytowy|niebyt|być|barowo|bytowy|bieżąco|bytowo|bycie|była|bardzo|bieżący|bar

dla człowieczo|ludzie|lud|człowieczy|ludowo|ludowy|człowiek stoa|stojąco|stanowo|stan|stojąca|stanowić|niestanie|stal|stale|stanąć|stała|stojący|stanowy|stać|niestojący|stały
dla ludzi stale
dla człowieka stała
dla człowieka stanem
dla człowieka stanowiły

rozwiązywać problem|problemowo|problemowy bar|bary|niebytowy|niebyt|być|barowo|bytowy|bieżąco|bytowo|bycie|była|bardzo|bieżący|barowy|bądź|były|niebieżący|byt|bieżeć
rozwiązywaniu problemu jest
rozwiązuje problem byłych
rozwiązywania problemów bieżącej
rozwiązywaniu problemów bytowych

oraz z bar|bary|niebytowy|niebyt|być|barowo|bytowy|bieżąco|bytowo|bycie|była|bardzo|bieżący|barowy|bądź|były|niebieżący|byt|bieżeć
oraz z bytom
oraz z bieżącego
oraz ze jest
oraz z bardzo

imiesłów|jedla|im|nie|jej|go|imienia|on|jedzenie|jadło|mu|nim|imiesłowowy|jet|jeść sytuacja bar|bary|niebytowy|niebyt|być|barowo|bytowy|bieżąco|bytowo|bycie|była|bardzo|bieżący|barowy|bądź|były|niebieżący|byt|bieżeć
ich sytuację jest
nie sytuację bieżącą
one sytuacji